<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
# from google.colab import drive
# import os
# import sys

# drive.mount("/content/drive", force_remount=True)
# try:
#     my_path = "/content/notebooks"
#     os.symlink("/content/drive/MyDrive/ColabNotebooks/my_env", my_path)
#     sys.path.insert(0, my_path)
# except:
#     print("Failed!")
# os.chdir("/content/drive/MyDrive/NLP")

import nltk
from glove import Corpus, Glove
import urllib
from lxml import etree
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
import json
import os
import sys

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82104\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [ ]:
(train_data, validation_data, test_data), info = tfds.load("imdb_reviews", split=["train[:80%]", "train[80%:90%]", "train[90%:]"], with_info=True, as_supervised=True)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
tf.keras.layers = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(tf.keras.layers, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
model = tf.keras.tf.keras.models()
model.add(hub_layer)
model.add(tf.keras.layers.tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
40/40 [==============================] - 3s 79ms/step - loss: 1.0590 - accuracy: 0.4897 - val_loss: 0.7653 - val_accuracy: 0.4656
Epoch 2/20
40/40 [==============================] - 3s 69ms/step - loss: 0.7244 - accuracy: 0.5088 - val_loss: 0.6995 - val_accuracy: 0.5448
Epoch 3/20
40/40 [==============================] - 3s 71ms/step - loss: 0.6714 - accuracy: 0.5929 - val_loss: 0.6558 - val_accuracy: 0.6228
Epoch 4/20
40/40 [==============================] - 3s 70ms/step - loss: 0.6317 - accuracy: 0.6512 - val_loss: 0.6214 - val_accuracy: 0.6648
Epoch 5/20
40/40 [==============================] - 3s 68ms/step - loss: 0.5965 - accuracy: 0.6873 - val_loss: 0.5880 - val_accuracy: 0.6968
Epoch 6/20
40/40 [==============================] - 3s 68ms/step - loss: 0.5611 - accuracy: 0.7211 - val_loss: 0.5517 - val_accuracy: 0.7220
Epoch 7/20
40/40 [==============================] - 3s 70ms/step - loss: 0.5224 - accuracy: 0.7526 - val_loss: 0.5142 - val_accuracy: 0.7508
Epoch 8/20
40

In [ ]:
corpuss = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, corpuss):
    print("%s: %.3f" % (name, value))

NameError: name 'model' is not defined